In [1]:
# File von Maren 11.04.2022
# Spaltfunktion h(x)=g(x)+delta(x)
# Integral für Dehnung ist eingebaut - delta wieder deaktiviert


In [2]:
import torch
import torchphysics as tp
import numpy as np
import pytorch_lightning as pl
from IPython.display import display, Math, Latex
import time
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False



In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Training on', device)
torch.cuda.is_available()

Training on cuda:0


True

In [4]:
#decide to train a new model or to load an existing one
train_new = True
#trained_model_name = '.pt'
trained_model_name = 'Trained_Model_Tribo.pt'
new_trained_model_name = 'Trained_Model_Tribo_new.pt'

In [5]:
#print('Tutorial zu TorchPhysics:')
#print('https://torchphysics.readthedocs.io/en/latest/tutorial/tutorial_start.html')

In [6]:
#display('Reynoldsgleichung (1.1) aus Multilevel Methods in Lubrication von C.H. Venner and A.A. Lubrecht')
#display(Math(r'u_m \frac{d}{d x} ( \rho h) - \frac{d}{ dx} \left( \frac{\rho h^3}{12 \nu} \
#   \frac{d}{d x} \right) = 0 \quad \mbox{with} \quad \rho=\rho(p)\,, \quad \nu=\nu(p)\,, \quad h=h(x) '))

In [7]:
# Konstanten für Stoffunktionen
beta = 2.2*1e-08 	# 2.2e-08 m^2/N
nu_0 = 15e-03 		# Pa·s = 1.5 mPa·s
# lower and upper bounds of parameters
# Viskosität [Pa s]


rho_0 = 800

# Geschwindigkeit [m/s]

um_0 = 2.0
# Abstand von Kugel-Platte [m]

#dh_0 = -1.51932e-6
dh_0 = 0.2e-6

pskal = 1e+7
# ------------------------------------------------
# Konstanten von Sergey
FN = 170000 # N hier ist Einheit wichtig!!
R = 0.0027 # m
E1 = 210*1e+9 # N/m2
v1 =0.3 

Er = E1/(1-v1*v1)

# Integrationsintervallgrenze
#IG = 35*np.sqrt((8*R*FN)/(Er*np.pi))
a = np.sqrt((8*R*FN)/(Er*np.pi))

lamb = pskal*a**3/(12*nu_0)

In [8]:
# define g_tilde: x/a, but also g/a**2
def g(x, dh):   
    
    out = dh/(a**2) + x**2/(2*R)
    return out

def g_x(x, dh):
    
    out = x/R
  
    return out

In [9]:
# define delta
# Verformung nach einem einfachen Mechanikgesetz

eps=1e-16

def delta(p_integral, x, x_integral):
    #Version mit oder ohne Skalierung durch a - Unterschied Verschiebung um Konstante
    #out = -4/(np.pi*Er)*6*a*torch.mean(pskal*p_integral*torch.log((torch.abs(x-x_integral)/a+eps)), dim=1)
    #out = -4/(np.pi*Er)*6*a*torch.mean(pskal*p_integral*torch.log(torch.abs(x-x_integral)+eps), dim=1)
    out = torch.zeros_like(x)
    return out

def delta_x(p_integral, x, x_integral):   
    #out = -4/(np.pi*Er)*6*a*torch.mean(pskal*p_integral/(torch.abs(x-x_integral)+eps)*torch.sgn(x-x_integral), dim=1)  
    out = torch.zeros_like(x)
    return out

In [10]:
# Spalthoehe entspricht h = g + delta 

In [11]:
#rho_tilde: rho/rho0
def rho_func(p):
    A = 5.9e+8
    B = 1.34   
    
    return (A+B*p*pskal)/(A+p*pskal)

In [12]:
# define rho_p
#rho_tilde: rho/rho0
def rho_p_func(p):
    A = 5.9e+8
    B = 1.34
    
    return ((B/(A+p*pskal)- (A+B*p*pskal)/((A+p*pskal)*(A+p*pskal))))

In [13]:
# Viskosität
# eta_tilde = eta/eta_0
def eta_func(p):
    out = torch.exp(beta * p*pskal)
    return torch.clamp(out, min=1e-06)


def eta_p_func(p): # Ableitung der Viskosität nach dem Druck p
    out = beta*torch.exp(beta*p*pskal)
    return out 

In [14]:
def penalty_p(p):
    #out = torch.nn.functional.relu(-p)*p**2
    pen_fac = 1e7
    out = pen_fac * torch.nn.functional.relu(-p)
    #out = p*pskal * torch.nn.functional.relu(-p)
    
    return out

In [15]:
# Variables:
x  = tp.spaces.R1('x')

# output
p = tp.spaces.R1('p')

In [16]:
#A_x  = tp.domains.Interval(x, -4*a, 2*a)
A_x  = tp.domains.Interval(x, -4, 2)


In [17]:
inner_sampler = tp.samplers.RandomUniformSampler(A_x, n_points = 15000) 

#inner_sampler = tp.samplers.AdaptiveRejectionSampler(A_x,n_points = 1000)

#integral_sampler = tp.samplers.RandomUniformSampler(A_x, n_points=100)



In [18]:
#tp.utils.scatter(nu*um*dh, inner_sampler, boundary_v_sampler)

In [19]:
import os
from activation_fn import AdaptiveActivationFunction
#model_path = 'Pilot1.pt'
a_fn = AdaptiveActivationFunction(torch.nn.Tanh())
model = tp.models.Sequential(
    tp.models.NormalizationLayer(A_x),
    tp.models.QRES(input_space=x, output_space=p,hidden=(30,30,30,30), 
                  activations=a_fn))#(50,50,50,50)))#,50,50,50,50)))
#if os.path.exists(model_path):
#    model.load_state_dict(torch.load(model_path))

In [20]:
#display(Math(r'h(x)\frac{d^2 \tilde{p}}{d x^2}  +\left( 3 \frac{dh}{dx} - \frac{h}{\nu} \frac{d \nu}{d x} \
#    \right) \frac{d \tilde{p}}{d x} = \frac{6 u_m \nu}{p_0 h^2} \frac{d h}{d x}\quad \mbox{with} \
#    \quad \tilde{p}=\frac{p}{p_{skal}} '))

In [21]:
def ansatz_func(x,p):    
    #return torch.where(x >-1, (2-x)*p, (x+4)*p) 
    #return 
    #return p
    return (2-x)*(x+4)*p/10.0
    #return (2-x)*(x+4)*torch.exp(-(x*a)**2)*p
    

In [22]:
from torchphysics.utils import grad
from torchphysics.utils import laplacian

def pde(p,  x): 
#def pde(p, p_integral, x, x_integral):     
   
   
    q=ansatz_func(x,p)
    
    #q_integral = ansatz_func(x_integral,p_integral)
    
    # evaluate the viscosity and their first derivative
    eta = eta_func(q)
    eta_p = eta_p_func(q) 
    # implement the PDE:  
    #  g und g_x mit Input dh:
    g_out = g(x, dh_0) # nur einmal auswerten
    g_x_out = g_x(x, dh_0) # nur einmal auswerten
    # delta und delta_p bereitstellen
    #delta_out = delta(q_integral, x, x_integral) # s.o.
    #delta_x_out = delta_x(q_integral, x, x_integral)
    
    h_out = g_out# + delta_out
    h_x_out = g_x_out# + delta_x_out
    # Dichtegradient bereitstellen
    rho_out = rho_func(q)
    rho_p_out = rho_p_func(q)
    
    s1 = -lamb
    s2 = um_0*pskal*h_out*rho_p_out - 3*lamb*h_out**2/(eta)*rho_out*h_x_out 
    s3 = lamb*(h_out**3)/eta*(eta_p*rho_out/eta - rho_p_out) 
    s4 = um_0*rho_out*h_x_out
    #print(s2)
   
    out =s1*grad(grad(q,x),x) + s2*grad(q,x) + s3*grad(q,x)**2 + s4
    return 1e-5*out

In [23]:
def positive_residual(p):
    return torch.clamp(p, max=0)

pos_sampler = tp.samplers.RandomUniformSampler(A_x, n_points = 3000) 

pos_condition = tp.conditions.PINNCondition(module=model,
                                            sampler=pos_sampler,
                                            residual_fn=positive_residual,   
                                            weight=100)

In [24]:
pde_condition = tp.conditions.PINNCondition(module=model,
                                                   sampler=inner_sampler,
                                                   residual_fn=pde,   
                                                   weight=1000,
                                                   #integral_sampler=integral_sampler,
                                                   name='pde_condition')

In [25]:
torch.cuda.is_available()

True

# Training des Modells oder laden 

In [26]:
if train_new:
    # zu LBFGS wechseln. 
    # hier LBFGS setzen
   #optim = tp.OptimizerSetting(optimizer_class=torch.optim.LBFGS, lr=0.1, 
     #                       optimizer_args={'max_iter': 2, 'history_size': 100})
    #optim = tp.solver.OptimizerSetting(torch.optim.Adam, lr=1e-5) #SGD, LBFGS
    optim = tp.solver.OptimizerSetting(torch.optim.Adam, lr=1e-3) #SGD, LBFGS
    solver = tp.solver.Solver([pde_condition, pos_condition],optimizer_setting = optim)
    #solver = tp.solver.Solver((pde_condition, boundary_condition),optimizer_setting = optim)

    trainer = pl.Trainer(gpus='-1' if torch.cuda.is_available() else None,
                     num_sanity_val_steps=0,
                     benchmark=True,
                     log_every_n_steps=1,
                     max_steps=5000,
                     logger=False,
                     checkpoint_callback=False
                     )

    trainer.fit(solver)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-GPU-9bdc43ab-3c7a-9d92-3ec5-4207d76cdb68/13/0]

  | Name             | Type       | Params
------------------------------------------------
0 | train_conditions | ModuleList | 5.6 K 
1 | val_conditions   | ModuleList | 0     
------------------------------------------------
5.6 K     Trainable params
0         Non-trainable params
5.6 K     Total params
0.023     Total estimated model params size (MB)
/home/krd2rng/.conda/envs/pytorch-physics/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/krd2rng/.conda/envs/pytorch-physics/lib/python3.9/site-pack

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [27]:
import matplotlib.pyplot as plt
print('Zwischenergebnis')
plot_sampler = tp.samplers.PlotSampler(plot_domain=A_x, n_points=600)
fig = tp.utils.plot(model,ansatz_func,plot_sampler) 


Zwischenergebnis


In [28]:
if True:
    # zu LBFGS wechseln. 
    # hier LBFGS setzen
    optim = tp.OptimizerSetting(optimizer_class=torch.optim.LBFGS, lr=0.5, 
                                optimizer_args={'max_iter': 4, 'history_size': 100})
    pde_condition.sampler = pde_condition.sampler.make_static()   
    pos_condition.sampler = pos_condition.sampler.make_static()
    solver = tp.solver.Solver([pde_condition, pos_condition],optimizer_setting = optim)

    trainer = pl.Trainer(gpus='-1' if torch.cuda.is_available() else None,
                     num_sanity_val_steps=0,
                     benchmark=True,
                     log_every_n_steps=1,
                     max_steps=1000,
                     logger=False,
                     checkpoint_callback=False
                     )

    trainer.fit(solver)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-GPU-9bdc43ab-3c7a-9d92-3ec5-4207d76cdb68/13/0]

  | Name             | Type       | Params
------------------------------------------------
0 | train_conditions | ModuleList | 5.6 K 
1 | val_conditions   | ModuleList | 0     
------------------------------------------------
5.6 K     Trainable params
0         Non-trainable params
5.6 K     Total params
0.023     Total estimated model params size (MB)
/home/krd2rng/.conda/envs/pytorch-physics/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/krd2rng/.conda/envs/pytorch-physics/lib/python3.9/site-pack

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [32]:
fig = tp.utils.plot(model,ansatz_func,plot_sampler) 

In [33]:
import numpy as np
data = np.genfromtxt('Referenzloesung_starr.CSV', delimiter=',')[1:, :]
plot_sampler = tp.samplers.PlotSampler(plot_domain=A_x, n_points=600, device='cpu')
fig = tp.utils.plot(model, ansatz_func, plot_sampler) 
plt.plot(data[:, 0]/a,data[:, 1]/pskal)
plt.legend(['Neural Network', 'Data'], loc='upper left')

In [34]:
a_fn.a

Parameter containing:
tensor(1.1799, requires_grad=True)